In [192]:
import sys
# %store -r SYSPATHS
SYSPATHS = {"tool":"../../../tool"}
sys.path.append(SYSPATHS['tool'])
import Khfeature as kf
import Khcore1 as kc1
from imp import reload
import MyEcharts as me
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn import linear_model
import pickle
import math

reload(kc1)
reload(kf)

<module 'Khfeature' from '../../../tool/Khfeature.py'>

In [193]:
step = "1min"  ## 计算步进间隔
## 选用三个 不同的时间尺度
temp_window = "15min"  ## 平均温度计算窗口: 平均温度的窗口尽量小, 体现其短时作用, 可以快速结束报警, 尺度也不能太短, 防止label毛刺 
roll_window = '2H'  ## 滑动计算窗口: 滑动窗口用于计算Q1,Q3,IQR 和 diff, 滑动窗口下的统计量, 时间可以稍微长点
slope_window = "3H"  ## 斜率计算窗口: 滑动温度斜率窗口, 因为温度存在小范围周期波动, 所以需要将时间窗口放大
th = 61.5  ## 报警温度阈值
rate = 0.55  ## 报警概率阈值

# 因变量模型效果验证

## 一、获取原始数据

In [194]:
### 1. 获取原始数据
names = ["TE_1003a","TE_1003b","TE_1003c"]
datas=  []
for name in names:
    datas.append(kc1.ReadRawData(name,month="2019_07"))
data = pd.concat(datas,axis=1)
data.columns = names
data['PT_4001'] = kc1.ReadRawData("PT_4001",month="2019_07",factory="中碳能源")
data['TE_1003'] = kc1.ReadRawData("TE_1003",column='value', factory='中碳能源', month='2019_07')
data = data.resample(step).mean()
data= data[data.index<"2019-07-12 00:00:00"]

## 二、自动打标签
    根据浓缩段平均温度 生成label

In [195]:
s2 = kf.SingleTsFeature(data['TE_1003'])
s2.add_label(th=th,left_window = '60min',right_window="1min",merge_window="20min",filename="label_auto")
label = s2.GetDf()[['label']]

save image as label_auto
add label successful 


../../../tool/Khfeature.py:109: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  right_window = pd.to_timedelta(right_window).total_seconds()/S
../../../tool/Khfeature.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  left = (left_window)*N-1


##### 查看自动标签效果

In [196]:
temp = pd.concat([data,label],axis=1)
temp['label'] = temp['label']*70
temp['PT_4001'] = temp['PT_4001']*100
me.Plot_LineBar(temp,name='Temp_raw_label_07',overwrite=True)

## 三、特征加工

### 1. 浓缩温度 temp : temp_window 下的 平均温度最大值
### 2. 浓缩温度 diff ： roll_window 下的 温度差值均值
### 3. 浓缩温度 iqr ：roll_window 下的 平均温度IQR
### 4. 浓缩温度 slope ：slope_window 下的 拟合斜率

In [197]:
### 1. 加工特征
s1 = kf.SingleTsFeature(data["TE_1003"]-th)  ##  将原始温度 减去 温度阈值 加工特征
s1.add_roll_quantile(q = 0.75,window=roll_window,varname = "q3")
s1.add_roll_quantile(q = 0.25,window=roll_window,varname = "q1")
s1.add_slope(window=slope_window)

add q3 successful 
add q1 successful 


../../../tool/Khfeature.py:200: FutureWarning: Currently, 'apply' passes the values as ndarrays to the applied function. In the future, this will change to passing it as Series objects. You need to specify 'raw=True' to keep the current behaviour, and you can pass 'raw=False' to silence this warning
  加工斜率特征


add_slope successful 


In [198]:
###  2. 某些特征 用户可自行生成
df = s1.GetDf()
df['iqr'] = df1['q3'] -df1['q1']
df['diff'] = data[['TE_1003a','TE_1003b','TE_1003c']].max(axis=1)-data['TE_1003']  ## 加工 温度差值 diff
df['diff'] = df['diff'].rolling(roll_window).mean()
df['temp'] = df['TE_1003'].rolling(temp_window).max()
df1 = pd.concat([df,label],axis=1)

df1['PT_4001'] = kc1.ReadRawData("PT_4001",month="2019_07",factory="中碳能源")
df1= df1[df1.index<"2019-07-12 00:00:00"].dropna()

In [199]:
me.Plot_LineBar(df1,columns = ["temp",'diff','iqr','slope','label'],name='Feature_label_07',overwrite=True)

## 三、模型预测 （调用6月model）

In [200]:
# scaler = pickle.load(open("./model/scaler.pkl","rb"))
model = pickle.load(open("./model/model.pkl","rb"))

In [206]:
model.coef_,model.intercept_

(array([[ 2.8438591 ,  0.75461285, -0.57411258, -1.05654281]]),
 array([1.50807295]))

In [203]:
columns = ["temp",'diff','iqr','slope']
X1= df1[columns].values
df1['Logistic06'] = model.predict_proba(X1)[:,1]

In [204]:
## 阈值规则修正label
## 将温度超过 th 的 label 直接置成报警概率
temp = df1.loc[df1[df1['TE_1003']>0].index,"Logistic06"]
df1.loc[temp[temp<rate].index,"Logistic06"] = rate
df1['rate'] = rate

In [205]:
me.Plot_LineBar(df1,columns = ['TE_1003','Logistic06','label','rate', 'PT_4001'],\
                name="Temp_Label_Logistic_07_0",overwrite=True)

## 四、模型预测 （7月训练model）

In [208]:
columns = ["temp",'diff','iqr','slope']
X1= df1[columns].values
Y = df1[['label']].values
model1 = linear_model.LogisticRegression(penalty = 'none',solver = 'saga', class_weight = 'balanced', max_iter=1000)
model1.fit(X1,Y)
print(model1.score(X1,Y))
print (model1.coef_,model1.intercept_)
df1['Logistic07'] = model1.predict_proba(X1)[:,1]

/root/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.9253740766462529
[[ 3.37198178  0.50056489 -0.43037506 -1.61088564]] [1.34295283]


In [209]:
me.Plot_LineBar(df1,columns = ['TE_1003','Logistic06','Logistic07','label','rate', 'PT_4001'],\
                name="Temp_Label_Logistic_07_1",overwrite=True)

In [211]:
## 固定模型参数
# def model3(df):
#     coef1 = pd.Series([49.00144768,3.57137631,-2.50972846, 3.1088844 ],index = ["temp",'diff','iqr','slope'])
#     coef2 = pd.Series([29.1605033,2.00337356,-1.28089264,  -2.10215003 ],index = ["temp",'diff','iqr','slope'])
#     coef = (coef1+coef2)/2.0
#     intercept_= (-23.27433937-15.72830235)/2.0
#     res = ((df[["temp",'diff','iqr','slope']]*coef).sum(axis=1)+intercept_)
#     res1 =res.map(lambda x:pow(math.e,x))
#     res2 = res1/(1+res1)
#     return res2.values